In [5]:
import sys
import os
import unittest
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [2]:
# add parent dir to the path and print the paths
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
print(f"Current directory: {current_dir}")
print(f"Parent directory: {parent_dir}")

sys.path.append(parent_dir)
print(f"sys.path: {sys.path}")

Current directory: c:\Users\ahaddadi\OneDrive\interv\project_mapping_cuco\soil_geochemistry\test
Parent directory: c:\Users\ahaddadi\OneDrive\interv\project_mapping_cuco\soil_geochemistry
sys.path: ['c:\\Users\\ahaddadi\\AppData\\Local\\anaconda3\\envs\\geochem\\python310.zip', 'c:\\Users\\ahaddadi\\AppData\\Local\\anaconda3\\envs\\geochem\\DLLs', 'c:\\Users\\ahaddadi\\AppData\\Local\\anaconda3\\envs\\geochem\\lib', 'c:\\Users\\ahaddadi\\AppData\\Local\\anaconda3\\envs\\geochem', '', 'c:\\Users\\ahaddadi\\AppData\\Local\\anaconda3\\envs\\geochem\\lib\\site-packages', 'c:\\Users\\ahaddadi\\AppData\\Local\\anaconda3\\envs\\geochem\\lib\\site-packages\\win32', 'c:\\Users\\ahaddadi\\AppData\\Local\\anaconda3\\envs\\geochem\\lib\\site-packages\\win32\\lib', 'c:\\Users\\ahaddadi\\AppData\\Local\\anaconda3\\envs\\geochem\\lib\\site-packages\\Pythonwin', 'c:\\Users\\ahaddadi\\OneDrive\\interv\\project_mapping_cuco\\soil_geochemistry']


In [7]:
# importing your modules
try:
    from utility_function.interpolations_methods import (
        create_gridx_gridy,
        interpolate_pnt_idw)
    from utility_function.victor_process import (
        find_minimum_rectangle_gdf,
        distance_nearest_neighbor_pnt)
    
    from utility_function.raster_process import(
        imshow_raster
    )
    
    
    print("Modules imported successfully.")
except ImportError as e:
    print(f"Error importing modules: {e}")

class TestGeospatialFunctions(unittest.TestCase):

    def setUp(self):
        # Create a sample GeoDataFrame for testing
        self.data = {
            'geometry': [Point(1, 1), Point(1, 20), Point(20, 1), Point(20, 20)],
            'value': [10, 20, 30, 40]
        }
        self.gdf = gpd.GeoDataFrame(self.data, crs="EPSG:3857")

    def test_create_gridx_gridy(self):
        grid_x, grid_y = create_gridx_gridy(self.gdf, cell_size=1, buffer=1)
        self.assertEqual(grid_x.shape[0], grid_y.shape[0])
        self.assertEqual(grid_x.shape[1], grid_y.shape[1])

    def test_interpolate_pnt_idw(self):
        result = interpolate_pnt_idw(self.gdf, 'value', cell_size=1, distance_coef=2, buffer=1)
        self.assertIn('Image', result)
        self.assertIn('GeoT_im', result)
        self.assertIn('ProjR_im', result)

    def test_find_minimum_rectangle_gdf(self):
        bnd_gdf = find_minimum_rectangle_gdf(self.gdf, buffer=1)
        self.assertEqual(len(bnd_gdf), 1)
        self.assertIsInstance(bnd_gdf.iloc[0].geometry, Polygon)

    def test_distance_nearest_neighbor_pnt(self):
        result = distance_nearest_neighbor_pnt(self.gdf)
        self.assertEqual(len(result), self.gdf.shape[0])


class TestRasterFuntions(unittest.TestCase):

    def setUp(self):
        # Create a dictionary 
        self.raster = {
            'Image': np.random.random((5,5)),
            'GeoT_im': (0, 1, 0, 5, 0, -1)          
        }
    def test_imshow_raster(self):
        result = imshow_raster(self.raster)
        



if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_create_gridx_gridy (__main__.TestGeospatialFunctions) ... ok
test_distance_nearest_neighbor_pnt (__main__.TestGeospatialFunctions) ... ok
test_find_minimum_rectangle_gdf (__main__.TestGeospatialFunctions) ... ok
test_interpolate_pnt_idw (__main__.TestGeospatialFunctions) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.113s

OK


Modules imported successfully.
{'Image': array([[20.03528425, 20.        , 20.04834925, 20.2218975 , 20.56397662,
        21.1139508 , 21.90121466, 22.94016726, 24.22703452, 25.73882111,
        27.43418453, 29.25581945, 31.13400034, 32.99111585, 34.74717523,
        36.32623366, 37.66338609, 38.71146846, 39.44612341, 39.86781775,
        40.        ],
       [20.06493364, 20.03870011, 20.09100176, 20.26325542, 20.59872438,
        21.13656275, 21.90580948, 22.92063191, 24.17758019, 25.65493211,
        27.31374462, 29.10010656, 30.9482244 , 32.78421794, 34.53072438,
        36.11248374, 37.46287795, 38.53086135, 39.28701233, 39.72699469,
        39.87103413],
       [20.14242793, 20.13803554, 20.20567144, 20.38508725, 20.71794785,
        21.24187508, 21.9846357 , 22.95976037, 24.1642167 , 25.57805861,
        27.16557655, 28.87741999, 30.65331633, 32.42524044, 34.12113264,
        35.66944514, 37.00476894, 38.07437891, 38.84473818, 39.30620699,
        39.4740774 ],
       [20.233942